<a href="https://colab.research.google.com/github/halen48/VisualG2C/blob/main/VisualG2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [409]:
import re
import inspect

In [410]:
nome_arq_c = ".c" #@param

#Programa

In [411]:
programas = []

In [412]:
programas.append("""
algoritmo "Calculos de numeros"
// Função : Exibe informações como o maior, menor número, a média e a sua soma
// Autor : Guilherme Novaes
// Data : 11/03/2022
// Seção de Declarações
var
numeros: vetor [1..5] de inteiro
teste1,teste2,teste3, teste4 : vetor [1..3,1..4] de real
nome : caractere
max, min,i,soma:inteiro
media : real
inicio
// Seção de Comandos

escreva("Digite seu nome: ")
leia(nome)

max<-numeros[1];
para i de 1 ate 5 faca
  escreva (nome, ", digite o ", i,"o valor: ")
  leia(numeros[i])

  // soma dos valores digitados
  soma <- soma + numeros[i];
  //media entre os valores digitados
  media<- soma / 5;

  se (numeros[i] > max) entao
    max<- numeros[i];
  fimse

  min <- numeros[1];
  se (numeros[i] < min) entao
    min <- numeros[i];
  fimse
fimpara

escreval ("maior numero: ", max)
escreval ("menor numero: ",min)
escreval("Soma dos numeros digitados:",soma)
escreva ("Média dos numero digitados:", media)

fimalgoritmo
""")

In [413]:
programas.append("""
algoritmo "Times"
var time: caractere
inicio
escreva ("Entre com o nome de um time de futebol: ")
leia (time)
escolha time
caso "Flamengo", "Fluminense", "Vasco", "Botafogo"
escreval ("É um time carioca.")
caso "São Paulo", "Palmeiras", "Santos", "Corínthians"
escreval ("É um time paulista.")
outrocaso
escreval ("É de outro estado.")
fimescolha
fimalgoritmo
""")

In [414]:
programas.append("""
algoritmo "Números de 1 a 10 (com interrompa)"
var x: inteiro
inicio
x <- 0
repita
x <- x + 1
escreva (x:3)
se x = 10 entao
interrompa
fimse
ate falso
fimalgoritmo
""")

#Parse

##Funções de Parse e Constantes

In [415]:
palavras_chave = {
    "inicio" : "fimalgoritmo",
    "se" : 'fimse',
    "para" : 'fimpara',
    "enquanto" : 'fimenquanto',
    "escolha" : "fimescolha",
    'repita' : 'ate',
}

In [416]:
palavras_chave_fechamento = {}
for palavra in palavras_chave:
  palavras_chave_fechamento[palavras_chave[palavra]] = palavra

In [417]:
tipos_variaveis = {
    'inteiro' : 'int',
    'real' : 'float',
    'caractere' : 'char',
    'logico' : 'bool',
}

In [418]:
mascaras_tipos_c = {
    'int' : '%d',
    'float' : '%f',
    'char' : '%s',
    'bool' : '%d',
}

```
Tamanho máximo da string que o visualG suporta
```

In [419]:
max_tam_string = 500 #@param {type:"integer"}

###Sub-parces

###Apelidos: Utilizados em um simples replace

In [420]:
apelidos = {
    '=' : '==',
    '<>' : '!=',
    '\\' : '/',
    'mod' : '%',
    'div' : '/',
    '<-' : '=',
    'limpatela' : '',
    'falso': 'false',
    'verdadeiro' : 'true',
    'arcsen' : 'asen',
    'arccos' : 'acos',
    'arctan' : 'atan',
    'cotan' : 'ctan',
    'logn' : 'log',
    'raizq' : 'sqrt',
    'exp' : 'pow',
    'caracpnum' : 'atoi',
    'numpcarac' : 'itoa',
    'compr'  : 'strlen',
    'copia'  : 'strcpy',
    'maiusc' : 'upper',
    'minusc' : 'lower',
}

###Regex

In [421]:
vetor_mask = re.compile(r'(.*): *vetor *\[(.*)\] * de * (\w+)')
var_mask = re.compile(r'(.*) *: *(\w+)')
potencia_mask = re.compile(r'([^\^]+)\^([^\-\+\*\\]+)') #pow([0],[1])
parametros_mask = re.compile(r'\(([^\)]*)\)')
para_mask = re.compile(r"para *([^ ]+) *de *([^ ]+) *ate *([^ ]+) faca")
se_mask = re.compile(r"se *(.*) *entao")
string_mask = re.compile(r'\"([^\"]+)\"')

###Utilidades

In [422]:
def eh_funcao(linha, parenteses = True):
  func_name = inspect.stack()[1].function.replace('parse_','')
  if(parenteses):
    try:
      params = parametros_mask.findall(linha)[0]
    except:
      raise Exception("[%s]: Erro de sintaxe em <%s>"%(func_name,linha))
  else:
    params = linha[len(func_name):]
  return (func_name == linha[:len(func_name)]),func_name, params

In [423]:
def nome_variavel(nome):
  idx = nome.find('[')
  if(idx > 0):
    return nome[:idx]
  return nome

###Casos especiais

####Base

In [424]:
def parse_inicio(linha, n_vars, t_vars):
  linha = linha.strip()
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  
  ret = """
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <string.h>
#include <math.h>
#include <time.h>
#include <ctype.h>

#define PI 3.141592

void upper(char * v){ int i; for(i = 0; i < strlen(v); i++) v[i] = toupper(v[i]); }
void lower(char * v){ int i; for(i = 0; i < strlen(v); i++) v[i] = tolower(v[i]); }

int main() {

\tsrand(time(NULL));
"""

  for tipo in t_vars:
    ret += '\t'+tipo + ' '
    for i in range(0,len(t_vars[tipo]), 5):
       ret += ', '.join(t_vars[tipo][i:i+5])
    ret += ';\n'
  
  ret += '\n'.join(['\tmemset(&%s, 0, sizeof(%s));'%(nome,nome) for nome in n_vars])

  return [ret+'\n']

In [425]:
def parse_fimalgoritmo(linha, _, __):
  linha = linha.strip()
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  return ["""
  return 0;
}
"""]

####Entrada & Saída

In [426]:
def parse_escreva(linha, n_vars, _):
  linha = linha.strip()
  eh,nome,params = eh_funcao(linha)
  if(not eh):
    return None
  pula_linha = ('escreval' == linha[:len('escreval')])
  printf_params = ""
  vars = []

  #primeiro isola as strings
  strs = string_mask.findall(params)
  for i,s in enumerate(strs):
    params = params.replace(s,'{%d}'%i)
  
  mask = None
  #agora realiza as outras operacoes
  for param in params.split(','):
    param = param.strip()
    nome = nome_variavel(param)
    if(':' in nome):
      nome = nome.split(':')
      nome,mask = nome[0],'.'.join(nome[1:])
      idx = param.find(':')
      param = param[:idx]
    if(nome in n_vars):
      if(mask):
        printf_params += '%'+mask+mascaras_tipos_c[n_vars[nome]].replace('%','')
      else:
        printf_params += mascaras_tipos_c[n_vars[nome]]
      vars.append(param)
    elif(param[0] == '"'):
      printf_params += strs[int(param[2:-2])]
    else:
      raise Exception("[%s]: Variavel <%s> não existe!!"%(nome,param))
  
  if(pula_linha):
    printf_params += '\\n'

  printf_params += '"'
  if(vars):
    printf_params += ', '+', '.join(vars)
  
  return [('printf("%s);'%printf_params)]

In [427]:
parse_escreval = parse_escreva

In [428]:
def parse_leia(linha, n_vars, _):
  linha = linha.strip()
  eh,_,params = eh_funcao(linha)
  if(not eh):
    return None
  
  scanf_params = ""
  vars = []

  for param in params.split(','):
    param = param.strip()
    nome = nome_variavel(param)
    if(nome in n_vars):
      scanf_params += mascaras_tipos_c[n_vars[nome]]
      
      if(n_vars[nome] == 'char'):
        vars.append(param)
      else:
        vars.append("&"+param)
    else:
      print("Variáveis do programa:")
      print(n_vars)
      raise Exception("[Leia]: Variavel %s não existe!!"%param)
  

  scanf_params += '"'
  if(vars):
    scanf_params += ', '+', '.join(vars)
  
  return [('scanf("%s);'%scanf_params)]

####Condicional

In [429]:
switches = []

In [430]:
def parse_escolha(linha, n_vars, _):
  global switches
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  switches.append({"Var" : None, "Tipo" : None, "LastOpen" : False})
  switch = switches[-1]
  switch["Var"] = linha.split(' ')[-1].strip()
  if(not switch["Var"] in n_vars):
    raise Exception("[Escolha]: Variável <%s> não encontrada!"%switch['Var'])
  switch["Tipo"] = n_vars[switch["Var"]]
  if(switch["Tipo"] == 'int'):
    return ["switch(%s)"%switch["Var"]]
  elif(switch["Tipo"] == 'float'):
    raise Exception("[Escolha]: Variável <%s> não pode ser do tipo real!!"%switch['Var'])
  else:
    return ["\n"]

In [431]:
def parse_caso(linha, n_vars, _):
  global switches
  eh,nome,params = eh_funcao(linha, False)
  if(not eh):
    return None
  switch = switches[-1]
  cases = params.split(',')
  ret = ""
  # ====================================
  if(switch["LastOpen"]):
    if(switch["Tipo"] == 'int'):
      ret = 'break;'
    else:
      ret = '} else '
  else:
    switch["LastOpen"] = True
  # ====================================
  if(switch["Tipo"] == 'int'):
    return [ret+"\n",'\n'.join(["case %s:"%case for case in cases])]
  else:
    return [ret+'if(%s) {'%(' || '.join(['(memcmp(%s,%s,sizeof(%s)) == 0)'%(switch['Var'], case, case) for case in cases]))]

In [432]:
def parse_outrocaso(linha,_,__):
  global switches
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  switch = switches[-1]
  if(switch['Tipo'] == 'int'):
    return ['default:']
  else:
    return ['} else {']

In [433]:
def parse_se(linha, n_vars, _):
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None

  try:
    exp = se_mask.findall(linha)[0]
  except:
    raise Exception("[Se]: Erro de sintaxe em <%s>"%linha)

  return [('if (%s) {'%(exp))]

####Laço

In [434]:
def parse_para(linha, n_vars, _):
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  
  try:
    var, inicio, fim = para_mask.findall(linha)[0]
  except:
    raise Exception("[Para]: Erro de sintaxe em <%s>"%linha)

  if(not nome_variavel(var) in n_vars):
    raise Exception ("[Para]: <%s> não declarada!!"%var)
  if(inicio > fim): #decrementa
    return [('for(%s = %s; %s >= %s; %s--) {'%(var, inicio, var, fim, var))]
  else:
    return [('for(%s = %s; %s <= %s; %s++) {'%(var, inicio, var, fim, var))]

In [435]:
def parse_ate(linha, n_vars, __):
  eh,nome,params = eh_funcao(linha, False)
  if(not eh):
    return None
  
  vars = {}
  for name in n_vars:
    vars[name] = 0
  try:
    eval(params.replace('false','False').replace('true','True'),{'__builtins__': None},vars)
  except:
    raise Exception("[Ate]: Erro de sintaxe em <%s> (Variável não existe?)"%linha)

  return [('} while(!(%s));'%(params))]

In [436]:
def parse_interrompa(linha,_,__):
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  return ['break;']

In [437]:
def parse_repita(linha,_,__):
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  return ['do {\n']

####Matemática

In [438]:
def parse_randi(linha,_,__):
  eh,_,params = eh_funcao(linha)
  if(not eh):
    return None
  return ['(rand()%%%s)'%params]

In [439]:
def parse_rand(linha,_,__):
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  return ['((rand()%32767)/32767.)']

In [440]:
def parse_int(linha,_,__):
  eh,_,params = eh_funcao(linha)
  if(not eh):
    return None
  return ['((int)(%s))'%params]

In [441]:
parse_asc = parse_int

In [442]:
def parse_grauprad(linha,_,__):
  eh,_,params = eh_funcao(linha)
  if(not eh):
    return None
  return ['(%s * (PI/180.))'%params]

In [443]:
def parse_asc(linha,_,__):
  eh,_,params = eh_funcao(linha)
  if(not eh):
    return None
  return ['(%s * (PI/180.))'%params]

###Outras funções

In [444]:
def limpa_comentario(codigo):
  codigo_final = ""

  ignora = False
  for idx in range(len(codigo)):
    if(ignora):
      if(codigo[idx] == '\n'):
        ignora = False
    else:
      if(codigo[idx:idx+2] == '//'):
          ignora = True
      else:
        codigo_final += codigo[idx]

  return codigo_final.replace('\t',' ')

In [445]:
def variaveis_globais(codigo):
  #=====================================================================
  def var_dict(v):
    if(not v in t_vars):
        t_vars[v] = []
  #=====================================================================
  t_vars = {} #tipo : nome
  n_vars = {} #nome : tipo
  escopo = codigo[:codigo.find('inicio')]
  for nomes_var, tipo in var_mask.findall(escopo):
  # ============ Variaveis simples =====================================
    nomes_var, tipo = nomes_var.lower().replace('var ',''), tipo.lower()
    if(tipo == 'vetor'):
      continue #Tradados depois...
    var_dict(tipos_variaveis[tipo])
    #Tipo caractere no visual G é uma string
    if(tipo == 'caractere'):
      t_vars[tipos_variaveis[tipo]] += [var.strip()+'[%d]'%max_tam_string for var in nomes_var.split(',')]
    else:
      t_vars[tipos_variaveis[tipo]] += [var.strip() for var in nomes_var.split(',')]
  # ============= Vetores ==============================================
  for nomes_var, dimensoes, tipo in vetor_mask.findall(escopo):
    tipo = tipo.lower()
    dimensoes = ''.join(['[%s]'%v.split('..')[-1] for v in dimensoes.split(',')])
    var_dict(tipos_variaveis[tipo])
    t_vars[tipos_variaveis[tipo]] += ["%s%s"%(v.strip().lower(),dimensoes) for v in nomes_var.split(',')]
    
  
  for tipo in t_vars:
    for nome in t_vars[tipo]:
      idx = nome.find('[')
      if(idx > 0):
        n_vars[nome[:idx]] = tipo
      else:
        n_vars[nome] = tipo
  
  return t_vars,n_vars

###Classe principal

In [446]:
class VisualG2C():
  def __init__(self, programa):
    self.__stack = []
    self.__programa = limpa_comentario(programa)
    self.t_vars,self.n_vars = variaveis_globais(self.__programa)

  def __parse_bloco(self, lin):
    lin = lin.strip()
    ret = None
    tokens = (
        lin
        .lower()
        .replace('(', ' ( ')
        .replace(')', ' ) ')
        .split(' ')
    )
    try:
      ret = eval("parse_%s(lin, self.n_vars, self.t_vars)"%tokens[0].lower())
    except NameError:
      ret = None
    nv = len(self.__stack)
    for token in tokens:
      if(token == 'ate' and 'para' in tokens):
        continue #Ignora esse...
      if(token in palavras_chave_fechamento):
        if(palavras_chave[self.__stack[-1]] == token):
          self.__stack.pop()
          nv -= 1
        else:
          print(self.__stack)
          print(tokens)
          raise Exception("[Bloco] Erro no delimitador <%s>"%token)
      if(token in palavras_chave):
        self.__stack.append(token)
    
    
    if(not ret and lin in palavras_chave_fechamento):
      ret = '}'
    return nv,ret
      

  def eval(self):
    idx = self.__programa.find('inicio')
    if(idx < 0):
      raise Exception("[Eval]: Programa não possui <inicio>!!!")

    for valor in apelidos:
      self.__programa = self.__programa.replace(valor," %s "%apelidos[valor])

    programa_C = [""]
    
    for lin in self.__programa[idx:].split('\n'):
      nv, n_lins = self.__parse_bloco(lin)
      
      if(n_lins is None):
        programa_C.append('\t'*(nv))
        programa_C[-1] += (lin)
        if(' = ' in lin):
          programa_C[-1] += ';'
      else:
        for p in n_lins:
          programa_C.append('\t'*(nv))
          
          if(('caso' in lin)):
            programa_C[-1] += len(switches)*'\b'
          programa_C[-1] += (p)
          
      
    if(len(self.__stack)):
      print(self.__stack)
      raise Exception("[Eval]: Erro de sintaxe em <%s>!!!"%self.__stack[-1])

    for linha in programa_C:
      print(linha) 
    programa_C = '\n'.join(programa_C).replace('\b','')

    return programa_C



#Exemplo

In [447]:
for i,programa in enumerate(programas):
  p = VisualG2C(programa)
  with open(("%03d"%i)+nome_arq_c, 'w') as f:
    f.write(p.eval())
  print("Programa %03d salvo!"%i)



#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <string.h>
#include <math.h>
#include <time.h>
#include <ctype.h>

#define PI 3.141592

void upper(char * v){ int i; for(i = 0; i < strlen(v); i++) v[i] = toupper(v[i]); }
void lower(char * v){ int i; for(i = 0; i < strlen(v); i++) v[i] = tolower(v[i]); }

int main() {

	srand(time(NULL));
	char nome[500];
	int max, min, i, soma, numeros[5];
	float media, teste1[3][4], teste2[3][4], teste3[3][4], teste4[3][4];
	memset(&nome, 0, sizeof(nome));
	memset(&max, 0, sizeof(max));
	memset(&min, 0, sizeof(min));
	memset(&i, 0, sizeof(i));
	memset(&soma, 0, sizeof(soma));
	memset(&numeros, 0, sizeof(numeros));
	memset(&media, 0, sizeof(media));
	memset(&teste1, 0, sizeof(teste1));
	memset(&teste2, 0, sizeof(teste2));
	memset(&teste3, 0, sizeof(teste3));
	memset(&teste4, 0, sizeof(teste4));

	
	printf("Digite seu nome: ");
	scanf("%s", nome);
	
	max = numeros[1];;
	for(i = 1; i <= 5; i++) {
		printf("%s, digite o %do valor: ",

In [448]:
for i in range(len(programas)):
  f_name = ("%03d"%i)+nome_arq_c
  !gcc $f_name -o teste
  print('\n')
  print("="*40)
  !./teste



Digite seu nome: Guilherme
Guilherme, digite o 1o valor: 3
Guilherme, digite o 2o valor: 6
Guilherme, digite o 3o valor: 23
Guilherme, digite o 4o valor: 1
Guilherme, digite o 5o valor: 5
maior numero: 23
menor numero: 3
Soma dos numeros digitados:38
Média dos numero digitados:7.000000

Entre com o nome de um time de futebol: Santos
É um time paulista.


  1  2  3  4  5  6  7  8  9 10